In [ ]:
# it isnot fully completed , just a test of algorithm

import numpy as np
import cv2
import pyrealsense2 as rs 

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
# Start streaming
pipeline.start(config)

im_name = 'fifa19'
font = cv2.FONT_ITALIC

# upload you file - enter file name / file must be in same folder as script
img = cv2.imread("fifa19.png", 0) # 0 = cv2.IMREAD_GRAYSCALE

cap = cv2.VideoCapture(0)

sift = cv2.xfeatures2d.SIFT_create()
kp_image, desc_image = sift.detectAndCompute(img, None)  # вместо None можно вставлять маску

#img = cv2.drawKeypoints(img,kp_image,img)                                                 ##query image

#Start feature matching
index_params = dict(algorithm=0, trees=5)
search_params = dict()
flann = cv2.FlannBasedMatcher(index_params, search_params)

while True:
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    depth_image = np.asanyarray(depth_frame.get_data())
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
    depth_colormap = cv2.drawKeypoints(depth_colormap, kp_image ,depth_colormap)
    
    _, frame = cap.read()
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # кадр видео тоже делаем серым    # train image
    
    kp_grayframe, desc_grayframe = sift.detectAndCompute(grayframe, None)  # кейпоинтс и дескриптор
    grayframe = cv2.drawKeypoints(grayframe,kp_grayframe,grayframe)  # рисуем кейпоинты и на видео картинке
    
    
    depth_colormap = cv2.drawKeypoints(depth_colormap, kp_grayframe ,depth_colormap)
    #feature matching
    matches=flann.knnMatch(desc_image,desc_grayframe,k=2)
    
    good_points=[]
    for m, n in matches:
        if m.distance < 0.5*n.distance:
            good_points.append(m)
    
    #img3 = cv2.drawMatches(img, kp_image, grayframe, kp_grayframe, good_points, grayframe)   # A to see knn match
    
    #Homografy for matching
    
    try:
        if len(good_points) > 5:
            query_pts = np.float32([kp_image[m.queryIdx].pt for m in good_points]).reshape(-1, 1, 2)
            train_pts = np.float32([kp_grayframe[m.trainIdx].pt for m in good_points]).reshape(-1, 1, 2)
 
            matrix, mask = cv2.findHomography(query_pts, train_pts, cv2.RANSAC, 5.0)
            matches_mask = mask.ravel().tolist()
        
            # Perspective transform
            h, w = img.shape
            pts = np.float32([[0, 0], [0, h], [w, h], [w, 0]]).reshape(-1, 1, 2)
        
            dst = cv2.perspectiveTransform(pts, matrix)
 
            homography = cv2.polylines(frame, [np.int32(dst)], True, (255, 0, 0), 3)    # 255 рисум синий
            # пишем на изображении
            #homography = cv2.putText(frame, im_name ,(20,400), font, 4,(255,0,0),2,cv2.LINE_AA)
    
            homography = np.hstack((homography, depth_colormap))
            
        
            cv2.imshow("Homography", homography)
        else:
            grayframe = np.hstack((grayframe, depth_colormap))
            cv2.imshow("Homography", grayframe)
    
        #cv2.imshow("Image", img)
        #cv2.imshow("Frame", grayframe)
        #cv2.imshow("compared", img3)                                                             # A to see knn match
    
        k = cv2.waitKey(30) & 0xFF
        if k == 27:
            break
    except:
        print ('Restarting!')
        continue
            
            
cap.release()
cv2.destroyAllWindows()
# Stop streaming
pipeline.stop()